In [ ]:
%cd ..

In [ ]:
from apperception.database import database

In [ ]:
############ ROAD DIRECTION ############

In [ ]:
### Add Road Direction Attributes to Tables ###
query = """ALTER TABLE Item_General_Trajectory
           ADD roadDirections tfloat;

           ALTER TABLE Cameras
           ADD roadDirection numeric;
        """
database._execute_query(query=query)
database._commit(True)

In [ ]:
### Update Road Direction For Camera ###
query = """UPDATE Cameras
           SET roadDirection = roadDirection(egoTranslation, egoHeading)
           WHERE roadDirection IS NULL;
        """
database._execute_query(query=query)
database._commit(True)

In [ ]:
### Create Function For roadDirection With Traj ###
query = """ 
        DROP FUNCTION IF EXISTS _roadDirectionAttr(trajectory tgeompoint, headings tfloat);
        CREATE OR REPLACE FUNCTION _roadDirectionAttr(trajectory tgeompoint, headings tfloat) RETURNS tfloat AS
        $BODY$
        declare trajString text;
        declare trajPointString text;
        declare trajPoint geometry;
        declare trajTime timestamptz;
        declare result tfloat;
        declare currentRoadDir float;
        declare currentRoadDirTime tfloat;
        BEGIN
        trajString := asText(trajectory);
        trajString := substr(trajString, 3, length(trajString) - 3);
        FOREACH trajPointString IN ARRAY STRING_TO_ARRAY(trajString, ',')
        LOOP
                trajPoint := ST_GeomFromText((STRING_TO_ARRAY(trajPointString, '@'))[1]);
                trajTime :=  (STRING_TO_ARRAY(trajPointString, '@'))[2];

                currentRoadDir := roadDirection(trajPoint, CAST(valueAtTimestamp(headings, trajTime) AS numeric));
                currentRoadDirTime := CAST(currentRoadDir AS text) || CAST('@' AS text) || trajTime;

                IF result IS NULL THEN
                        result = currentRoadDirTime;
                ELSE
                        result := appendInstant(result, currentRoadDirTime);
                END IF;
        END LOOP;
        RETURN result;
        END
        $BODY$
        LANGUAGE 'plpgsql' ;
        """
database._execute_query(query=query)
database._commit(True)

In [ ]:
### Update Road Direction For General Trajectory Table ###
query = """UPDATE Item_General_Trajectory
           SET roadDirections = _roadDirectionAttr(trajCentroids, itemHeadings)
           WHERE roadDirections IS NULL AND objectType LIKE 'vehicle%';
        """
database._execute_query(query=query)
database._commit(True)

In [ ]:
############ MIN AND MAX TIMESTAMPS ############

In [ ]:
### Add minTimestamp Attributes to Trajectory Table ###
query = """ALTER TABLE Item_General_Trajectory
           ADD minTimestamp timestamptz;

           ALTER TABLE Item_General_Trajectory
           ADD maxTimestamp timestamptz;
        """
database._execute_query(query=query)
database._commit(True)

In [ ]:
### Update minTimestamp For Trajectories ###
query = """UPDATE Item_General_Trajectory
           SET minTimestamp = lower(trajCentroids::period)
           WHERE minTimestamp IS NULL;
        """
database._execute_query(query=query)
database._commit(True)

In [ ]:
### Update maxTimestamp For Trajectories ###
query = """UPDATE Item_General_Trajectory
           SET maxTimestamp = upper(trajCentroids::period)
           WHERE maxTimestamp IS NULL;
        """
database._execute_query(query=query)
database._commit(True)

In [ ]:
############ ROAD SEGMENT TYPES ############